# Function for Cluster Plot w/ Basic Linear Regression
## Example Parameter: Avg Leaf Nitrogen Concentration (LNC) vs Leaf Carbon Nitrogen Ratio (LCN)
#### Author: Heather Childers, Sofia Ingersoll, Sujan Bhattaria 
##### Date: 2024-02-18

##### Loading environment settings

In [3]:
# moved the libraries that were here into utils.py because they're essential
# xarray is required to run the utils import line
import xarray as xr
import glob

In [4]:
# import libraries & data pre-processing functions from utils.py
from utils import *

ModuleNotFoundError: No module named 'utils'

##### Request additional processing power from server

In [3]:
# Request an additional 10 cores of power for processing from the server
client = get_cluster("UCSB0021", cores = 40)

In [4]:
# apply peer2peer network communication across multiple devices
client.cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/sbhattarai/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.118:39321,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/sbhattarai/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


### By loading utils,

A dummy data array `da` & data set `ds` were loaded into our local environment. This is a pre-processed data, the array is of the data variable of interest: LNC: leaf nitrogen content. 

In addition to the dummy data, the functions defined in the utils.py library are also now accesible in this notebook and my be utilized to wrangle the cluster data that is to be read in.

In [5]:
# LNC data array read in with utils library
da

<xarray.DataArray 'LNC' (time: 120, gridcell: 400)>
[48000 values with dtype=float32]
Coordinates:
  * time     (time) object 2005-02-01 00:00:00 ... 2015-01-01 00:00:00
Dimensions without coordinates: gridcell
Attributes:
    long_name:     leaf N concentration
    units:         gN leaf/m^2
    cell_methods:  time: mean

In [6]:
# data set read in with utils library
ds

<xarray.Dataset>
Dimensions:                            (levgrnd: 25, levsoi: 20, levlak: 10,
                                        levdcmp: 25, time: 120,
                                        hist_interval: 2, lon: 144, lat: 96,
                                        gridcell: 400, landunit: 1148,
                                        column: 2599, pft: 4387, cft: 2,
                                        ltype: 9, nvegwcs: 4)
Coordinates:
  * levgrnd                            (levgrnd) float32 0.01 0.04 ... 42.0
  * levsoi                             (levsoi) float32 0.01 0.04 ... 6.94 8.03
  * levlak                             (levlak) float32 0.05 0.6 ... 34.33 44.78
  * levdcmp                            (levdcmp) float32 0.01 0.04 ... 42.0
  * time                               (time) object 2005-02-01 00:00:00 ... ...
  * lon                                (lon) float32 0.0 2.5 5.0 ... 355.0 357.5
  * lat                                (lat) float32 -90.0 -88.11 ... 88.11 90.0
Dimensions without coordinates: hist_interval, gridcell, landunit, column, pft,
                                cft, ltype, nvegwcs
Data variables: (12/497)
    mcdate                             (time) int32 ...
    mcsec                              (time) int32 ...
    mdcur                              (time) int32 ...
    mscur                              (time) int32 ...
    nstep                              (time) int32 ...
    time_bounds                        (time, hist_interval) object ...
    ...                                 ...
    XSMRPOOL                           (time, gridcell) float32 ...
    XSMRPOOL_RECOVER                   (time, gridcell) float32 ...
    ZBOT                               (time, gridcell) float32 ...
    ZWT                                (time, gridcell) float32 ...
    ZWT_CH4_UNSAT                      (time, gridcell) float32 ...
    ZWT_PERCH                          (time, gridcell) float32 ...
Attributes: (12/40)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   created on 09/06/23 02:33:14
    source:                                    Community Terrestrial Systems ...
    hostname:                                  cheyenne
    ...                                        ...
    ctype_urban_pervious_road:                 75
    cft_c3_crop:                               1
    cft_c3_irrigated:                          2
    time_period_freq:                          month_1
    Time_constant_3Dvars_filename:             ./PPEn11_CTL2010_OAAT0000.clm2...
    Time_constant_3Dvars:                      ZSOI:DZSOI:WATSAT:SUCSAT:BSW:H...

## Loading a Cluster of 500 files
The data files are located in `/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/`

The 2005-2010 monthly output files have the form:
`PPEn11_transient_LHC0001.clm2.h0.2005-02-01-00000.nc`

We're interested in files spanning from:
`LHC0001 to LHC0500`

In [53]:
#modify the function if you want to pass the parameter
def read_all_simulation(var):
    '''prepare cluster list and read to create ensemble(group of data)
    use preprocess to select only certain dimension and a variable'''
    # read all simulations as a list
    cluster_list= sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2005-02-01-00000.nc'))
    cluster_list = cluster_list[1:len(cluster_list)]

    # only select latitude, longitude, time, and  using this in preprocess steps
    def preprocess(ds, var):
        '''using this function in xr.open_mfdataset as preprocess
        ensures that when only these four things are selected 
        before the data is combined'''
        return ds[['lat', 'lon', 'time', var]]
    
    #read the list and load it for the notebook
    xr.open_mfdataset( cluster_list, 
                                   combine='nested',
                                   preprocess = lambda ds: preprocess(ds, var),
                                   parallel=False, 
                                   concat_dim="ens")



In [55]:
help(read_all_simulation)

Help on function read_all_simulation in module __main__:

read_all_simulation(var)
    prepare cluster list and read to create ensemble(group of data)
    use preprocess to select only certain dimension and a variable



##### Accessing data processing functions from utils.py library

In [2]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----    Wrangle  Cluster Data     ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Subset for the variable of interest LNC and properly weigh grid cells and time dim
# these functions are defined in the utils library 
cluster_ds = cluster_ds['LNC'].weighted_landarea_gridcells(landarea).yearly_weighted_average().fix_time()

NameError: name 'cluster_ds' is not defined

### Leaf Carbon : Nitrogen Data

This will come in handy later when we want to plot trends overtime

In [ ]:
# Leaf CN data for plotting
df = pd.read_csv('/glade/campaign/asp/djk2120/PPEn11/csvs/lhc220926.txt',index_col=0)
# convert to data set
params = xr.Dataset(df)

# the only dimension here is the 'member' aka file index id [LCH0001-500]
params

# subsetting for leafcn
leafcn = params['leafcn']

#leafcn

### Visualize Cluster Plot with Linear Regression

**NOTE: For right now we're using the dummy data until we're able to load in the cluster files**

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----     Subsetting Dummy Data    ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# only the da was pre_processed, so we're weighing the ds here and making an array
leafcn_da = ds['LEAFCN'].dropna('gridcell')

leafcn_da

**This is not working for our time dimension on leafcn_da**

In [ ]:
# only the da was pre_processed, so we're weighing the ds here and making an array
leafcn_da = yearly_weighted_average(leafcn_da)
#.fix_time()

In [ ]:
da

In [ ]:
# Creating Linear Regression for LNC

# using utils pre-processed data array for LNC
# split into training & testing (90/10)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#from sklearn_xarray import Stacker, Select


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Split Data 90/10        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
da = da.dropna('gridcell')

X_train, X_test, y_train, y_test = train_test_split(leafcn_da, da, test_size = 0.1, random_state=0)


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Linear Regression       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

y_pred = regressor.predict(X_test)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----    Plot Linear Regression    ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
plt.scatter(X_train, y_train,color='g')
plt.plot(X_test, y_pred,color='k')

plt.show()

In [ ]:
# calculate global mean over time
LNC_avg = da.mean(dim = 'time')

# plot mean tsa overtime of weighted gridcells
plt.figure(figsize = (9,4))
plt.scatter(leafcn_da,
            LNC_avg,
            c = LNC_avg,
           cmap = 'RdPu',
           alpha = 0.5)
plt.xlabel('Leaf Carbon Nitrogen Ratio')
plt.ylabel('Leaf Nitrogen Concentration')
plt.title('Default Simulation: OAAT0000')
plt.colorbar(label = 'Concenctration (units to be added)')

### Down sampled 2 file approach

In [ ]:
# Set filepath
#filepath = '/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/'

In [ ]:
#members = ["LHC" + str(i).zfill(4) for i in range(1,501)]
#members

In [ ]:
#Open multiple files as a single dataset
ds_mf =xr.open_mfdataset(['/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC0001.clm2.h0.2005-02-01-00000.nc', 
                          '/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC0002.clm2.h0.2005-02-01-00000.nc'], 
                         combine='nested', parallel=True, concat_dim = "ens")

In [ ]:
ds_mf.time

In [ ]:
ds.time.max()

In [ ]:
file2 = '/glade/campaign/cgd/tss/projects/PPE/helpers/sparsegrid_landarea.nc'
ds2 = xr.open_dataset(file2)
landarea = ds2['landarea']
weighted_avg_area = ds_mf['TSA'].weighted(landarea).mean(dim = 'gridcell').mean(dim = 'time')
weighted_avg_area.values

#### Single file visualization of LNC

In [ ]:
# these are commented out because the utils library provides the same information in line 2
#file = "/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC0001.clm2.h0.2005-02-01-00000.nc"
#ds = xr.open_dataset(file)

In [ ]:
lnc = ds['LNC']
lnc_timeavg = lnc.mean(dim = 'time')

In [ ]:
plt.scatter(ds.grid1d_lon,
            ds.grid1d_lat,
            c=lnc_timeavg)

In [ ]:
#lnc_avg = lnc_timeavg.mean(dim = 'lat')